<a href="https://colab.research.google.com/github/kuriatsu/learning_genetic_algorithms/blob/main/ga_2023rwdc_shokki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.8 MB/s eta 0:00:00


# 必要なモジュールのインポート  
遺伝的アルゴリズムの計算には、DEAPを使用  
pprintは辞書型のprintをファンシーにしてくれる

* 公式ドキュメント
https://deap.readthedocs.io/en/master/index.html  
* 日本語説明ブログ
https://dse-souken.com/2021/05/25/ai-19/  
* Github
https://github.com/DEAP/deap/tree/60913c5543abf8318ddce0492e8ffcdf37974d86

In [5]:
import random
import numpy as np
from pprint import pprint

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# 変数定義
DAY : 計画期間（30日間の配送拠点を計画する）
LIFT_NUM : リフトの数

リフトのデータベース  
* id : リフトID
* base : 拠点
* capacity : 燃料タンク容量[L]
* cosumption : 1日ごとののリフト消費量予測(リフト03 : 毎日3L消費)
* initial_remaining : 計画期間初日の燃料残量
* remaining : N日目の燃料残量、GA計算中にN日目の燃料残量をシミュレーションするために使用する変数

In [6]:
DAY=30

lifts = [
  {
    "id":"03",
    "base": "kariya",
    "capacity": 60,
    "consumption": [3]*DAY,
    "initial_remaining": 60,
    "remaining": 60,
  },
  {
    "id":"30",
    "base": "takahama",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 40,
    "remaining": 40,
  },
  {
    "id":"52",
    "base": "takahama",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
  {
    "id":"19",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 30,
    "remaining": 30,
  },
  {
    "id":"32",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
]

LIFT_NUM=len(lifts)

# DEAPモデル構築  
遺伝的アルゴリズムの肝となる、親世代から子世代への遺伝方法は、いくつか選択肢があるので、調査してよりよいものを試してみてください
* トーナメント方式の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/selection.py
* 交叉関数の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/crossover.py
* 突然変異関数の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/mutation.py

## サンプルの内約  
```python
[0, 1, 10, 0, ...]
 |-> 00000 = 1日目、どこにも配送しない
    |-> 00001 = 2日目、"lifts"の0番目のリフト(03)へ給油
       |-> 01010 = 3日目、"lifts"の1と3番目のリフトへ給油
```

In [7]:
## 最小化問題として定義するので、-1.0を与える
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
## 1計画サンプルの定義、リスト内に30日分の配送計画が格納される
creator.create("Individual", list, fitness=creator.FitnessMin)

## GAに必要な各種関数の定義
toolbox = base.Toolbox()
## 1日の配送計画の値の生成方法(各リフトへの給油の組み合わせ=2^LIFT_NUM)
toolbox.register("attribute", random.randint, 0, 2**LIFT_NUM)
## 各サンプルに含まれる30日分の計画をattributeによって決定することを登録
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, DAY)
## サンプル集団の個体数を決定するための関数
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## トーナメント方式で次世代に子を残す親を選択, サンプル集団内で上から5番めまでのサンプルを次の世代に残す
toolbox.register("select", tools.selTournament, tournsize=5)
## 交叉関数を設定。2点交叉を採用
toolbox.register("mate", tools.cxTwoPoint)
## 突然変異関数の設定。0.2の確率で次の世代で突然変異を起こす。変異の幅(low-up)はサンプルが取りうる値の幅
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2**LIFT_NUM, indpb=0.2)

# 評価関数
## モデル
* 毎日予測した消費量`consumption`に従ってリフトのタンクの燃料`remaining`が減っていく
* 給油されると満タンになる（`remaining = capacity`）

## 評価
* １拠点に配送すると+10
* リフトの残燃料が２日分を切ると+1000
* この点数（`reward`）を最小化するように`DAY`日分の配送計画を立てる

## 実装できていない部分
* 配送拠点リストに対する配送順序と、消費燃料の評価が行われていない（各拠点ごとに+10してるだけ）
* 給油時間、配送時間は目的関数や制約に含まれていない
* トラックの積載上限は考慮していない（各リフトのタンク容量が90L程度で、トラック積載量が4000Lとかなので、余裕？）


In [8]:
def evalPlan(individual):
  """各サンプルの評価関数
  ~args~
  individual : １サンプル
  ~returns~
  reward : 評価関数の値(末尾に,が必要らしい)
  """
  reward = 0 ## この値を最小化

  ## リフトの残燃料を初期化
  for lift_index, lift in enumerate(lifts):
    lift["remaining"] = lift["initial_remaining"]

  ## 配送とリフトの残燃料をシミュレート
  for day, supply_plan_in_a_day in enumerate(individual):
    delivery = set() ## １日の配送拠点
    time = 0 ## 配送にかかった時間[min] (まだ何にも使用してない)

    for lift_index, lift in enumerate(lifts):
      ## 残燃料が2日分(今日と明日分)以下になったらペナルティ
      if (day+1 < DAY-1 and lift["remaining"] - lift["consumption"][day] - lift["consumption"][day+1] < 0) or \
         (day   > 0       and lift["remaining"] - lift["consumption"][day-1] - lift["consumption"][day] < 0):
        reward += 1000

      ## 燃料を消費
      lift["remaining"] -= lift["consumption"][day]

      ## day日目の給油計画にlift番目のリフトが存在するかチェック(ビット演算)
      if supply_plan_in_a_day & (1<<lift_index):
        delivery.add(lift["base"]) ## 給油拠点をリストに追加
        time += 0.03 * (lift["capacity"] - lift["remaining"]) ## 給油時間を計算
        lift["remaining"] = lift["capacity"] ## 燃料満タン

    ## 1日の配送拠点に対するペナルティ
    for delivery_base in delivery:
      reward += 10 # 拠点へ配送した際のペナルティ(適当)
      time += 90 # 配送時間90分(適当)

  return reward,

toolbox.register("evaluate", evalPlan)

# 最適化開始
パレートフロントを解としているが、多目的最適化問題のための関数な気がする。。。  
初期サンプル10000とかだと結構時間かかる  
100とかだとあまり最適化されない？  

In [16]:
random.seed(64)

pop = toolbox.population(n=3000) ## 最初にnサンプル作成
hof = tools.ParetoFront() ## どのように最適解を得るかを決定

## 各世代のサンプルのrewardに対する統計情報を表示する
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

## 最適化開始、ngen世代まで生成
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg   	std    	min	max 
0  	3000  	597.93	228.996	420	7550
1  	1764  	534.167	97.8112	410	4430
2  	1822  	507.28 	254.4  	370	9490
3  	1833  	466.367	149.471	340	5500
4  	1838  	441.07 	218.414	320	8410
5  	1800  	418.26 	361    	280	15360
6  	1834  	387.487	257.543	260	8340 
7  	1817  	380.973	380.945	220	8290 
8  	1844  	369.547	492.948	210	10260
9  	1817  	349.957	538.839	180	14260
10 	1782  	421.283	1038.06	170	20200
11 	1825  	400.887	1051.33	160	21170
12 	1790  	499.69 	1391.88	140	20200
13 	1786  	461.257	1236.47	130	19280
14 	1823  	568.917	1833.43	120	22130
15 	1819  	595.707	1903.33	110	21170
16 	1754  	585.543	1929.4 	100	21140
17 	1831  	752.143	2362.36	90 	21140
18 	1789  	869.727	2927.69	80 	33080
19 	1783  	942.49 	3103.7 	70 	29070
20 	1791  	1034.69	3238.75	70 	32100
21 	1740  	1037.14	3350.39	50 	38060
22 	1790  	1275.28	4039.23	50 	40060
23 	1809  	1378.82	4615.24	50 	41060
24 	1828  	1806.92	5511.13	40 	57050
25 	1875  	1735.47	5430.35	40 	45040
26 	1795 

# 結果出力

In [17]:
def decodePlan(individual):
  """配送計画のデコード
  ~args~
  individual : サンプル
  ~returns~
  plan : DAY日分の配送計画、給油対象のリフト
  """
  plan = {}
  for day, supply_plan_in_a_day in enumerate(individual):
    delivery = set()
    supply_lift = []
    for lift_index, lift in enumerate(lifts):
      if supply_plan_in_a_day & (1<<lift_index):
        delivery.add(lift["base"])
        supply_lift.append(lift["id"])
    plan[f"day_{day+1}"] = {"base":delivery, "lift": supply_lift}
  return plan

In [18]:
## 最適なサンプルの抽出
best_plan = tools.selBest(pop, 1)[0]
## サンプルをデコード
plan = decodePlan(best_plan)

## 表示
print("the best plan is ")
pprint(plan)
print(f"penalty: {best_plan.fitness.values}")

the best plan is 
{'day_1': {'base': set(), 'lift': []},
 'day_10': {'base': set(), 'lift': []},
 'day_11': {'base': set(), 'lift': []},
 'day_12': {'base': set(), 'lift': []},
 'day_13': {'base': set(), 'lift': []},
 'day_14': {'base': set(), 'lift': []},
 'day_15': {'base': set(), 'lift': []},
 'day_16': {'base': set(), 'lift': []},
 'day_17': {'base': set(), 'lift': []},
 'day_18': {'base': {'kariya'}, 'lift': ['03']},
 'day_19': {'base': set(), 'lift': []},
 'day_2': {'base': set(), 'lift': []},
 'day_20': {'base': set(), 'lift': []},
 'day_21': {'base': set(), 'lift': []},
 'day_22': {'base': set(), 'lift': []},
 'day_23': {'base': set(), 'lift': []},
 'day_24': {'base': set(), 'lift': []},
 'day_25': {'base': set(), 'lift': []},
 'day_26': {'base': set(), 'lift': []},
 'day_27': {'base': set(), 'lift': []},
 'day_28': {'base': set(), 'lift': []},
 'day_29': {'base': set(), 'lift': []},
 'day_3': {'base': set(), 'lift': []},
 'day_30': {'base': set(), 'lift': []},
 'day_4': {'base